In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
from sklearn import metrics


BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
import utils.metrics
import utils.record

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]

print(RANDOM_SEED)

PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')

RESULTS_DIR = os.path.join(BASE_DIR, 'results')

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, DATASET, ".pkl")
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base")

# models saved here
if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: no model has been done for rs={RANDOM_SEED}")

55


In [4]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "val.csv"))
hyper_val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_val.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DIR, "test.csv"))

In [6]:
x_val = val_df.drop('label', axis=1).values
y_val = val_df['label'].values

x_hyper_val = hyper_val_df.drop('label', axis=1).values
y_hyper_val = hyper_val_df['label'].values

x_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_val.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])

In [8]:
model_types = [
    "base",
    "ft",
    "lrw",
    "kmm",
    "jv"
]

metric_list = [
    'G-mean',
]

In [9]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)

Results file exists, appending to it...


In [10]:
for mt in model_types:
    print(f"Model Type: {mt}")
    modelpath = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt=mt)
    model.load_weights(modelpath)
    preds_val = utils.utils.compute_preds(
        model,
        x_val,
        batch_size=BATCH_SIZE,
    )
    preds_hyper_val = utils.utils.compute_preds(
        model,
        x_hyper_val,
        batch_size=BATCH_SIZE,
    )
    preds_test = utils.utils.compute_preds(
        model,
        x_test,
        batch_size=BATCH_SIZE,
    )

    preds_val = np.argmax(preds_val, axis=1)
    preds_hyper_val = np.argmax(preds_hyper_val, axis=1)
    preds_test = np.argmax(preds_test, axis=1)

    val_conf = metrics.confusion_matrix(y_val, preds_val, normalize='all')
    hyper_val_conf = metrics.confusion_matrix(y_hyper_val, preds_hyper_val, normalize='all')
    test_conf = metrics.confusion_matrix(y_test, preds_test, normalize='all')
    
    for metric in metric_list:
        val_score = utils.metrics.eval_metric(val_conf, metric)
        hyper_val_score = utils.metrics.eval_metric(hyper_val_conf, metric)
        test_score = utils.metrics.eval_metric(test_conf, metric)
        
        saver.save(RANDOM_SEED, metric, mt, val_score, hyper_val_score, test_score, None)

Model Type: base
Model Type: ft


In [11]:
saver.close()